# analogy: default program

In [1]:
import sys
sys.path.append("../")
from default import *
import os
import numpy as np

## Run the default solution on dev

In [2]:
!pip install gensim

In [3]:
import gensim.downloader as api
from gensim.models import KeyedVectors

Download the glove 100d model trained on the gigaword corpus.

In [4]:
model_gigaword = api.load("glove-wiki-gigaword-100")

Get the output on the word analogy task by reading the dev set and using vector arithmetic over the word vectors for the first 10 lines in the dev set.

In [5]:
output = []
not_working = []
working = []
with open(os.path.join('../data', 'input', 'dev.txt')) as fh:
    for i, line in enumerate(fh):
        if i>10:
            break
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        print(f"{a.lower()}-{b.lower()}+{c.lower()}={results[0][0]}")
        output.append(results[0][0])
print(output)

athens-greece+iraq=baghdad
athens-greece+thailand=bangkok
athens-greece+china=beijing
athens-greece+germany=berlin
athens-greece+switzerland=zurich
athens-greece+egypt=cairo
athens-greece+australia=sydney
athens-greece+vietnam=hanoi
athens-greece+cuba=havana
athens-greece+finland=helsinki
[': capital-common-countries', 'baghdad', 'bangkok', 'beijing', 'berlin', 'zurich', 'cairo', 'sydney', 'hanoi', 'havana', 'helsinki']


## Evaluate the default output

In [6]:
from analogy_check import precision
with open(os.path.join('../data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=80.00


## Documentation

Write some beautiful documentation of your program here.

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

In [7]:
# Generating GT's for test file

In [13]:

# output = {}
# inputs = {}
# with open(os.path.join('../data', 'input', 'test.txt')) as fh:
#     for i, line in enumerate(fh):
#         line = line.strip()
#         if line[0] == ':':
#             get_pairs = {}
#             output[line] = get_pairs
#             ints = []
#             inputs[line] = ints
#             continue
#         (a, b, c) = line.split()
#         get_pairs[a]=b
#         ints.append((a,b,c))
# sels = [': capital-world', ': gram6-nationality-adjective',': gram7-past-tense',': gram8-plural',': gram9-plural-verbs']
# oo = {': capital-world':[], ': gram6-nationality-adjective':[],': gram7-past-tense':[],': gram8-plural':[],': gram9-plural-verbs':[]}
# capital_world_out = []
# for sel in sels:
#     for _,_,ech in inputs[sel]:
#         ref_dict =output[sel]
#         if ech in ref_dict:
#             capital_world_out.append(ref_dict[ech])
#         else:
#             swapped_dict = {value: key for key, value in ref_dict.items()}
#             oo[sel].append(swapped_dict[ech])
        
# with open('../data/reference/test.txt', 'w') as file:
#     for sel in sels:
#         file.write(f"{sel}\n")
#         test_gt = oo[sel]
#         for item in test_gt:
#             file.write(f"{item}\n")

In [9]:
# Analysing Failure cases

In [14]:
pds = []
with open('../output/test.out', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        pds.append(line)
gts = []
with open('../data/reference/test.out', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        gts.append(line)
inp = []
with open('../data/input/test.txt', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        inp.append(line)

In [15]:
cor = 0
for i in range(len(inp)):
    if pds[i].strip() == gts[i].strip().lower():
        cor +=1
    else:
        print("(",inp[i].strip(),")", pds[i].strip(), gts[i].strip())

( Abuja Nigeria Madagascar ) conakry Antananarivo
( Abuja Nigeria Kazakhstan ) almaty Astana
( Abuja Nigeria Greece ) corfu Athens
( Abuja Nigeria Belize ) tegucigalpa Belmopan
( Abuja Nigeria Germany ) bonn Berlin
( Abuja Nigeria Switzerland ) zurich Bern
( Abuja Nigeria Burundi ) arusha Bujumbura
( Abuja Nigeria Australia ) brisbane Canberra
( Abuja Nigeria Moldova ) skopje Chisinau
( Accra Ghana Madagascar ) conakry Antananarivo
( Accra Ghana Kazakhstan ) almaty Astana
( Accra Ghana Belize ) tegucigalpa Belmopan
( Accra Ghana Switzerland ) zurich Bern
( Accra Ghana Australia ) perth Canberra
( Accra Ghana Ireland ) belfast Dublin
( Accra Ghana Tuvalu ) iqaluit Funafuti
( Algiers Algeria Madagascar ) conakry Antananarivo
( Algiers Algeria Kazakhstan ) almaty Astana
( Algiers Algeria Belize ) tegucigalpa Belmopan
( Algiers Algeria Switzerland ) zurich Bern
( Algiers Algeria Australia ) melbourne Canberra
( Algiers Algeria Moldova ) skopje Chisinau
( Algiers Algeria Qatar ) manama Doha

In [16]:
cor/len(inp)

0.7347496206373293

In [17]:
a,b,c = "run","ran", "sing"
model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])

[('perform', 0.6210198402404785),
 ('sang', 0.5608733892440796),
 ('tune', 0.5531637668609619),
 ('sung', 0.5515801310539246),
 ('sings', 0.5453202128410339),
 ('pray', 0.5444638729095459),
 ('singing', 0.5441002249717712),
 ('do', 0.5438512563705444),
 ('hear', 0.538349986076355),
 ('listen', 0.5350242257118225)]

In [18]:
import re
import copy
isNumber = re.compile(r'\d+.*')

def norm_word(word):
  if isNumber.search(word.lower()):
    return '---num---'
  elif re.sub(r'\W+', '', word) == '':
    return '---punc---'
  else:
    return word.lower()
      
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r'):
    words = line.lower().strip().split()
    lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
  return lexicon

In [19]:
lexicons = read_lexicon("../data/train/train/from_wikipedia.txt")

In [20]:
Q_hat = {word: model_gigaword[word] for word in model_gigaword.index_to_key}
Q = copy.deepcopy(Q_hat)

In [23]:
print("lexicons : ",len(lexicons.keys()))
print("Q_hat : ",len(Q_hat.keys()))
found = set(lexicons.keys()).intersection(set(Q_hat.keys()))
not_found = set(lexicons.keys()).difference(set(Q_hat.keys()))
print("found : ", len(lexicons.keys())-len(not_found))
print("not found : ", len(not_found))

lexicons :  474
Q_hat :  400000
found :  474
not found :  0


In [24]:
Q_hat_keys = set(Q_hat.keys())
epoch = 10
alpha = 1
beta = 1

for ep in range(epoch):
    print("epoch : ",ep)
    for idx,word in enumerate(found):
        print(idx,end='\r')
        word_synset = set(lexicons[word]).intersection(Q_hat_keys)
        if len(word_synset) == 0:
            continue
            
        qi = np.array([Q[ws] for ws in word_synset])
        sum_qi = np.sum(qi,axis = 0)
    
        Q[word] = ((beta*sum_qi) + (alpha*Q_hat[word]))/((beta*len(qi))+alpha)
    
    
    

epoch :  0
epoch :  1
epoch :  2
epoch :  3
epoch :  4
epoch :  5
epoch :  6
epoch :  7
epoch :  8
epoch :  9


In [7]:
def save_word_vectors(word_vecs, filename):
    num_words = len(word_vecs.keys())
    vector_size = len(word_vecs['the'])
    
    with open(filename, 'w') as f:
        f.write(f"{num_words} {vector_size}\n")
        
        for word in word_vecs.keys():
            vector = word_vecs[word]
            vector_str = ' '.join(map(str, vector))
            f.write(f"{word} {vector_str}\n")

In [26]:
save_word_vectors(Q,"./retrofit_new_vectors.txt")

In [27]:
output = []
not_working = []
working = []

retrofitted_model = KeyedVectors.load_word2vec_format('./retrofit_new_vectors.txt', binary=False)

with open(os.path.join('../data', 'input', 'dev.txt')) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        # results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        results = retrofitted_model.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        output.append(results[0][0])

with open(os.path.join('../data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=71.43


In [28]:
output = []
not_working = []
working = []

with open(os.path.join('../data', 'input', 'test.txt')) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        # results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        results = retrofitted_model.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        output.append(results[0][0])

with open(os.path.join('../data','reference','test.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=92.60


In [350]:
# wordnet-synonyms+.txt
Train = 13.92
Test = 41

# wordnet-synonyms+.txt (10)
Train = 14.93
Test = 43.63

# wordnet-synonyms.txt (10)
Train = 18.72
Test = 53.05

# ppdb-xl.txt (10)
Train = 24.33
Test = 47.23

# framenet.txt (10)
Train = 16.44
Test = 54.70

# from_wikipedia.txt (10)
Train = 71.42
Test =  92.60